## 5.2 Abandoned Lock
In the previous example, the critical section for this program exists between the acquire methods on lines 14 and 15, and the release methods on lines 21 and 22. If one of the philosopher's threads acquires the locks, and when something goes wrong in that critical section to cause an unexpected error, that could kill its thread before it gets a chance to release the lock. To simulate happening, We'll add another `if` statement to check it there're exactly 10 pieces of sushi left and do my favorite technique for intentionally crashing a program - divide by 0. 

We should never divide by 0, but we're doing it here to trigger an exception that will cause Python to crash the currently executing thread.

In [2]:
#!/usr/bin/env python3
""" Three philosophers, thinking and eating sushi """

import threading

chopstick_a = threading.Lock()
chopstick_b = threading.Lock()
chopstick_c = threading.Lock()
sushi_count = 500

def philosopher(name, first_chopstick, second_chopstick):
    global sushi_count
    while sushi_count > 0: # eat sushi until it's all gone
        first_chopstick.acquire()
        second_chopstick.acquire()

        if sushi_count > 0:
            sushi_count -= 1
            print(name, 'took a piece! Sushi remaining:', sushi_count)
            
        if sushi_count == 10:
            print(1/0)

        second_chopstick.release()
        first_chopstick.release()
        
if __name__ == '__main__':
    threading.Thread(target=philosopher, args=('Barron', chopstick_a, chopstick_b)).start()
    threading.Thread(target=philosopher, args=('Olivia', chopstick_b, chopstick_c)).start()
    threading.Thread(target=philosopher, args=('Steve', chopstick_a, chopstick_c)).start()


Barron took a piece! Sushi remaining: 499
Barron took a piece! Sushi remaining: 498
Barron took a piece! Sushi remaining: 497
Barron took a piece! Sushi remaining: 496
Barron took a piece! Sushi remaining: 495
Barron took a piece! Sushi remaining: 494
Barron took a piece! Sushi remaining: 493
Barron took a piece! Sushi remaining: 492
Barron took a piece! Sushi remaining: 491
Barron took a piece! Sushi remaining: 490
Barron took a piece! Sushi remaining: 489
Barron took a piece! Sushi remaining: 488
Barron took a piece! Sushi remaining: 487
Barron took a piece! Sushi remaining: 486
Barron took a piece! Sushi remaining: 485
Barron took a piece! Sushi remaining: 484
Barron took a piece! Sushi remaining: 483
Barron took a piece! Sushi remaining: 482
Barron took a piece! Sushi remaining: 481
Barron took a piece! Sushi remaining: 480
Barron took a piece! Sushi remaining: 479
Barron took a piece! Sushi remaining: 478
Barron took a piece! Sushi remaining: 477
Barron took a piece! Sushi remaini

Olivia took a piece! Sushi remaining: 51
Olivia took a piece! Sushi remaining: 50
Olivia took a piece! Sushi remaining: 49
Olivia took a piece! Sushi remaining: 48
Olivia took a piece! Sushi remaining: 47
Olivia took a piece! Sushi remaining: 46
Olivia took a piece! Sushi remaining: 45
Olivia took a piece! Sushi remaining: 44
Olivia took a piece! Sushi remaining: 43
Olivia took a piece! Sushi remaining: 42
Olivia took a piece! Sushi remaining: 41
Olivia took a piece! Sushi remaining: 40
Olivia took a piece! Sushi remaining: 39
Olivia took a piece! Sushi remaining: 38
Olivia took a piece! Sushi remaining: 37
Olivia took a piece! Sushi remaining: 36
Olivia took a piece! Sushi remaining: 35
Olivia took a piece! Sushi remaining: 34
Olivia took a piece! Sushi remaining: 33
Olivia took a piece! Sushi remaining: 32
Olivia took a piece! Sushi remaining: 31
Olivia took a piece! Sushi remaining: 30
Olivia took a piece! Sushi remaining: 29
Olivia took a piece! Sushi remaining: 28
Olivia took a pi

Exception in thread Thread-7:
Traceback (most recent call last):
  File "C:\An\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\An\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-cfeeed9ad6e0>", line 22, in philosopher
    print(1/0)
ZeroDivisionError: division by zero



When we run this program, it gets all the way down to 10 pieces remaining. Then the thread that happens to be executing at that time, in this case, that's Olivia thread, it hits that divide by zero and crashes. The other threads are stuck waiting on the locks that will never get released by Olivia, so the program is stuck here forever. 

This scenario is not the same as deadlock, because the threads here are not waiting on each other to release a lock. But it's a related scenario and the impact is the same: the program isn't making any progress. 


To prevent this type of situation from occurring, we should put the critical section within a `try` block. If we have any exception handling code, we can optionally include an `except` clause after the `try` block to catch and deal with that error. But what we really care about here is making sure that the locks always get released before the current thread gets terminated if it crashes. To do that, We'll also add a `finally` block after the `try` statement, and put the calls to unlock the chopsticks in it.

In [10]:
#!/usr/bin/env python3
""" Three philosophers, thinking and eating sushi """

import threading

chopstick_a = threading.Lock()
chopstick_b = threading.Lock()
chopstick_c = threading.Lock()
sushi_count = 500

def philosopher(name, first_chopstick, second_chopstick):
    global sushi_count
    while sushi_count > 0: # eat sushi until it's all gone
        first_chopstick.acquire()
        second_chopstick.acquire()
        
        try:
            if sushi_count > 0:
                sushi_count -= 1
                print(name, 'took a piece! Sushi remaining:', sushi_count)

            if sushi_count == 10:
                print(1/0)
                
        finally:
            second_chopstick.release()
            first_chopstick.release()
        
if __name__ == '__main__':
    threading.Thread(target=philosopher, args=('Barron', chopstick_a, chopstick_b)).start()
    threading.Thread(target=philosopher, args=('Olivia', chopstick_b, chopstick_c)).start()
    threading.Thread(target=philosopher, args=('Steve', chopstick_a, chopstick_c)).start()


Barron took a piece! Sushi remaining: 499
Barron took a piece! Sushi remaining: 498
Barron took a piece! Sushi remaining: 497
Barron took a piece! Sushi remaining: 496
Barron took a piece! Sushi remaining: 495
Barron took a piece! Sushi remaining: 494
Barron took a piece! Sushi remaining: 493
Barron took a piece! Sushi remaining: 492
Barron took a piece! Sushi remaining: 491
Barron took a piece! Sushi remaining: 490
Barron took a piece! Sushi remaining: 489
Barron took a piece! Sushi remaining: 488
Barron took a piece! Sushi remaining: 487
Barron took a piece! Sushi remaining: 486
Barron took a piece! Sushi remaining: 485
Barron took a piece! Sushi remaining: 484
Barron took a piece! Sushi remaining: 483
Barron took a piece! Sushi remaining: 482
Olivia took a piece! Sushi remaining: 481
Olivia took a piece! Sushi remaining: 480
Olivia took a piece! Sushi remaining: 479
Olivia took a piece! Sushi remaining: 478
Olivia took a piece! Sushi remaining: 477
Olivia took a piece! Sushi remaini

Olivia took a piece! Sushi remaining: 148
Olivia took a piece! Sushi remaining: 147
Olivia took a piece! Sushi remaining: 146
Olivia took a piece! Sushi remaining: 145
Olivia took a piece! Sushi remaining: 144
Olivia took a piece! Sushi remaining: 143
Olivia took a piece! Sushi remaining: 142
Olivia took a piece! Sushi remaining: 141
Olivia took a piece! Sushi remaining: 140
Olivia took a piece! Sushi remaining: 139
Olivia took a piece! Sushi remaining: 138
Olivia took a piece! Sushi remaining: 137
Olivia took a piece! Sushi remaining: 136
Olivia took a piece! Sushi remaining: 135
Olivia took a piece! Sushi remaining: 134
Olivia took a piece! Sushi remaining: 133
Olivia took a piece! Sushi remaining: 132
Olivia took a piece! Sushi remaining: 131
Olivia took a piece! Sushi remaining: 130
Olivia took a piece! Sushi remaining: 129
Olivia took a piece! Sushi remaining: 128
Olivia took a piece! Sushi remaining: 127
Olivia took a piece! Sushi remaining: 126
Olivia took a piece! Sushi remaini

Exception in thread Thread-19:
Traceback (most recent call last):
  File "C:\An\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\An\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-10-5537972dec6c>", line 23, in philosopher
    print(1/0)
ZeroDivisionError: division by zero



In [11]:
#!/usr/bin/env python3
""" Three philosophers, thinking and eating sushi """

import threading

chopstick_a = threading.Lock()
chopstick_b = threading.Lock()
chopstick_c = threading.Lock()
sushi_count = 500

def philosopher(name, first_chopstick, second_chopstick):
    global sushi_count
    while sushi_count > 0: # eat sushi until it's all gone
        first_chopstick.acquire()
        second_chopstick.acquire()

        try:
            if sushi_count > 0:
                sushi_count -= 1
                print(name, 'took a piece! Sushi remaining:', sushi_count)

            if sushi_count == 10:
                print(1/0)

        finally:
            second_chopstick.release()
            first_chopstick.release()
        
if __name__ == '__main__':
    threading.Thread(target=philosopher, args=('Barron', chopstick_a, chopstick_b)).start()
    threading.Thread(target=philosopher, args=('Olivia', chopstick_b, chopstick_c)).start()
    threading.Thread(target=philosopher, args=('Steve', chopstick_a, chopstick_c)).start()


Barron took a piece! Sushi remaining: 499
Barron took a piece! Sushi remaining: 498
Barron took a piece! Sushi remaining: 497
Barron took a piece! Sushi remaining: 496
Barron took a piece! Sushi remaining: 495
Barron took a piece! Sushi remaining: 494
Barron took a piece! Sushi remaining: 493
Barron took a piece! Sushi remaining: 492
Barron took a piece! Sushi remaining: 491
Barron took a piece! Sushi remaining: 490
Barron took a piece! Sushi remaining: 489
Barron took a piece! Sushi remaining: 488
Barron took a piece! Sushi remaining: 487
Barron took a piece! Sushi remaining: 486
Barron took a piece! Sushi remaining: 485
Barron took a piece! Sushi remaining: 484
Barron took a piece! Sushi remaining: 483
Barron took a piece! Sushi remaining: 482
Barron took a piece! Sushi remaining: 481
Barron took a piece! Sushi remaining: 480
Barron took a piece! Sushi remaining: 479
Barron took a piece! Sushi remaining: 478
Barron took a piece! Sushi remaining: 477
Barron took a piece! Sushi remaini

Olivia took a piece! Sushi remaining: 64
Olivia took a piece! Sushi remaining: 63
Olivia took a piece! Sushi remaining: 62
Olivia took a piece! Sushi remaining: 61
Olivia took a piece! Sushi remaining: 60
Olivia took a piece! Sushi remaining: 59
Olivia took a piece! Sushi remaining: 58
Olivia took a piece! Sushi remaining: 57
Olivia took a piece! Sushi remaining: 56
Olivia took a piece! Sushi remaining: 55
Olivia took a piece! Sushi remaining: 54
Olivia took a piece! Sushi remaining: 53
Olivia took a piece! Sushi remaining: 52
Olivia took a piece! Sushi remaining: 51
Olivia took a piece! Sushi remaining: 50
Olivia took a piece! Sushi remaining: 49
Olivia took a piece! Sushi remaining: 48
Olivia took a piece! Sushi remaining: 47
Olivia took a piece! Sushi remaining: 46
Olivia took a piece! Sushi remaining: 45
Olivia took a piece! Sushi remaining: 44
Olivia took a piece! Sushi remaining: 43
Olivia took a piece! Sushi remaining: 42
Olivia took a piece! Sushi remaining: 41
Olivia took a pi

Exception in thread Thread-22:
Traceback (most recent call last):
  File "C:\An\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\An\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-11-7464641c155c>", line 23, in philosopher
    print(1/0)
ZeroDivisionError: division by zero



This time, an `exception` still occurs when one of the threads takes the 10th piece of sushi, in this case, that was Olivia, but thanks to the `finally` clause, that thread is able to release the lock before it terminates. We can see that after the Olivia thread took the 10th piece of sushi and crashed, the Barron thread took over to finish eating the remaining sushi.

It's good practice to always make sure lock will be released if something goes wrong and unexpectedly crashes a thread. Python makes that especially easy because `lock` objects support working with context managers. Using the `with` statement on a lock is equivalent to using the `try` and `finally` blocks. Using a context manager is a more pythonic way to program.

```
some_lock.acquire()
try:
    # do something...
finally:
    some_lock.release()
    
# More pythonic structured
with some_lock:
    # do something...
```

In [13]:
#!/usr/bin/env python3
""" Three philosophers, thinking and eating sushi """

import threading

chopstick_a = threading.Lock()
chopstick_b = threading.Lock()
chopstick_c = threading.Lock()
sushi_count = 500

def philosopher(name, first_chopstick, second_chopstick):
    global sushi_count
    while sushi_count > 0: # eat sushi until it's all gone
        with first_chopstick:
            with second_chopstick:
                if sushi_count > 0:
                    sushi_count -= 1
                    print(name, 'took a piece! Sushi remaining:', sushi_count)

                if sushi_count == 10:
                    print(1/0)

if __name__ == '__main__':
    threading.Thread(target=philosopher, args=('Barron', chopstick_a, chopstick_b)).start()
    threading.Thread(target=philosopher, args=('Olivia', chopstick_b, chopstick_c)).start()
    threading.Thread(target=philosopher, args=('Steve', chopstick_a, chopstick_c)).start()


Barron took a piece! Sushi remaining: 499
Barron took a piece! Sushi remaining: 498
Barron took a piece! Sushi remaining: 497
Barron took a piece! Sushi remaining: 496
Barron took a piece! Sushi remaining: 495
Barron took a piece! Sushi remaining: 494
Barron took a piece! Sushi remaining: 493
Barron took a piece! Sushi remaining: 492
Barron took a piece! Sushi remaining: 491
Barron took a piece! Sushi remaining: 490
Barron took a piece! Sushi remaining: 489
Barron took a piece! Sushi remaining: 488
Barron took a piece! Sushi remaining: 487
Barron took a piece! Sushi remaining: 486
Barron took a piece! Sushi remaining: 485
Barron took a piece! Sushi remaining: 484
Barron took a piece! Sushi remaining: 483
Barron took a piece! Sushi remaining: 482
Barron took a piece! Sushi remaining: 481
Barron took a piece! Sushi remaining: 480
Barron took a piece! Sushi remaining: 479
Barron took a piece! Sushi remaining: 478
Barron took a piece! Sushi remaining: 477
Barron took a piece! Sushi remaini

Olivia took a piece! Sushi remaining: 85
Olivia took a piece! Sushi remaining: 84
Olivia took a piece! Sushi remaining: 83
Olivia took a piece! Sushi remaining: 82
Olivia took a piece! Sushi remaining: 81
Olivia took a piece! Sushi remaining: 80
Olivia took a piece! Sushi remaining: 79
Olivia took a piece! Sushi remaining: 78
Olivia took a piece! Sushi remaining: 77
Olivia took a piece! Sushi remaining: 76
Olivia took a piece! Sushi remaining: 75
Olivia took a piece! Sushi remaining: 74
Olivia took a piece! Sushi remaining: 73
Olivia took a piece! Sushi remaining: 72
Olivia took a piece! Sushi remaining: 71
Olivia took a piece! Sushi remaining: 70
Olivia took a piece! Sushi remaining: 69
Olivia took a piece! Sushi remaining: 68
Olivia took a piece! Sushi remaining: 67
Olivia took a piece! Sushi remaining: 66
Olivia took a piece! Sushi remaining: 65
Olivia took a piece! Sushi remaining: 64
Olivia took a piece! Sushi remaining: 63
Olivia took a piece! Sushi remaining: 62
Olivia took a pi

Exception in thread Thread-25:
Traceback (most recent call last):
  File "C:\An\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\An\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-13-8f175e8947ee>", line 21, in philosopher
    print(1/0)
ZeroDivisionError: division by zero

